In [ ]:
!pip install ipywidgets

In [1]:
from .autonotebook import tqdm as notebook_tqdm

ImportError: attempted relative import with no known parent package

In [1]:
# from docling.chunking import HybridChunker
from langchain_docling import DoclingLoader
from langchain_docling.loader import ExportType

from docling.backend.docling_parse_backend import DoclingParseDocumentBackend
from docling.datamodel.base_models import InputFormat
from docling.datamodel.pipeline_options import (
    EasyOcrOptions,
    OcrMacOptions,
    PdfPipelineOptions,
    RapidOcrOptions,
    TesseractCliOcrOptions,
    TesseractOcrOptions,
    AcceleratorOptions,
    AcceleratorDevice
)
from docling.datamodel.settings import settings
from docling.document_converter import DocumentConverter, PdfFormatOption

from docling.chunking import HybridChunker

import os

/home/alexandre_pantalena/desenvolvimento/repos/langchain/langchain_venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
OCR = False


do_ocr: bool = OCR

do_table_structure: bool = True
generate_page_images: bool = True
generate_picture_images: bool = True

accelerator_options = AcceleratorOptions(
    num_threads=8, device=AcceleratorDevice.CPU
)

do_cell_matching: bool = True

pipeline_options = PdfPipelineOptions(
    do_ocr=do_ocr,
    do_table_structure=do_table_structure,
    generate_page_images=generate_page_images,
    generate_picture_images=generate_picture_images,
    accelerator_options = accelerator_options,
)
pipeline_options.table_structure_options.do_cell_matching = do_cell_matching

force_full_page_ocr: bool = OCR

# Any of the OCR options can be used:EasyOcrOptions, TesseractOcrOptions, TesseractCliOcrOptions, OcrMacOptions(Mac only), RapidOcrOptions
# ocr_options = EasyOcrOptions(force_full_page_ocr=force_full_page_ocr)
# ocr_options = TesseractOcrOptions(force_full_page_ocr=force_full_page_ocr)
# ocr_options = OcrMacOptions(force_full_page_ocr=force_full_page_ocr)
# ocr_options = RapidOcrOptions(force_full_page_ocr=force_full_page_ocr)
ocr_options = TesseractCliOcrOptions(force_full_page_ocr=force_full_page_ocr, lang=["Latin"])
pipeline_options.ocr_options = ocr_options

# Enable the profiling to measure the time spent
settings.debug.profile_pipeline_timings = True

doc_converter = DocumentConverter(
    format_options={
        InputFormat.PDF: PdfFormatOption(pipeline_options=pipeline_options)
    }
)

In [ ]:
FILE_PATH_ASSEMBLEIA = '../files/BRPVSCCRI3Y2-ASS31012025V01-000829323.pdf'
FILE_PATH_ASSEMBLEIA_NAO_INSTALADA = '../files/BRASTECRI0C3-ASS04022025V01-000830908.pdf'
FILE_PATH_ASSEMBLEIA_OCR = '../files/23A1772203-ASS04022025V01-000831063.pdf'
FILE_PATH_TS = '../files/831141.pdf'
FILE_PATH_TS_2 = '../files/24F1234491-TDS07062024V01-000675608.pdf'
FILE_PATH_TS_CRA = '../files/797444.pdf'
FILT_PATH_TS_3 = '../files/018759000101010.pdf'

FILE_PATH_ESCRITURA = '../files/676706.pdf'

FILE_PATH = FILT_PATH_TS_3

EMBED_MODEL_ID = "sentence-transformers/all-MiniLM-L6-v2"

loader = DoclingLoader(
    converter=doc_converter,
    file_path=FILE_PATH,
    export_type=ExportType.MARKDOWN,
    chunker=HybridChunker(tokenizer=EMBED_MODEL_ID),
)

docs = loader.load()

In [5]:
len(docs)

1

In [6]:
docs[0].page_content

'<unknown>\n\n## TERMO DE SECURITIZAÇÃO DE DIREITOS CREDITÓRIOS DO AGRONEGÓCIO\n\npara emissão de\n\n## CERTIFICADOS DE RECEBÍVEIS DO AGRONEGÓCIO DA 1ª E 2ª SÉRIES DA 34ª (TRIGÉSIMA QUARTA) EMISSÃO DA\n\nCERES SECURITIZADORA S.A.\n\ncomo companhia securitizadora\n\nlastreados em direitos creditórios do agronegócio devidos pela NATIVA AGRONEGÓCIOS LTDA.\n\ncelebrado com\n\nOLIVEIRA TRUST DISTRIBUIDORA DE TÍTULOS E VALORES MOBILIÁRIOS S.A. como agente fiduciário\n\nDatado de 18 de dezembro de 2023\n\nDocumento assinado no Assinador Registro de Imóveis. Para validar o documento e suas assinaturas acesse https://assinador.registrodeimoveis.org.br/validate/546KQ-TBSM7-VTY3M-6YDA8.\n\nTERMO  DE  SECURITIZAÇÃO  DE  DIREITOS  CREDITÓRIOS  DO  AGRONEGÓCIO  PARA  EMISSÃO  DE CERTIFICADOS DE RECEBÍVEIS DO AGRONEGÓCIO, DA 1ª E 2ª SÉRIES, DA 34ª (TRIGÉSIMA QUARTA) EMISSÃO  DA  CERES  SECURITIZADORA  S.A.,  LASTREADOS  EM  DIREITOS  CREDITÓRIOS  DO AGRONEGÓCIO DEVIDOS PELA NATIVA AGRONEGÓCIOS LTDA.\

In [7]:
from langchain_text_splitters import MarkdownHeaderTextSplitter

splitter = MarkdownHeaderTextSplitter(
    headers_to_split_on=[
        ("#", "Header_1"),
        ("##", "Header_2"),
        # ("###", "Header_3"),
    ],
)
splits = [split for doc in docs for split in splitter.split_text(doc.page_content)]

In [8]:
len(splits)

118

In [9]:
splits[0].page_content

'<unknown>'

In [10]:
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import AzureChatOpenAI
from pydantic import BaseModel, Field, model_validator

# deployment_name = "gpt-4o-mini"
# embedding_model = "gpt-4o-mini"
# openai_api_version="2024-07-18"

system_prompt = ("""
Você é um assistente de IA que extrai dados de documentos.
Você deve extrair informações específicas e retorná-las em um formato estruturado.

INSTRUÇÕES CRÍTICAS:
- EVITE DUPLICADOS: Nunca inclua itens duplicados em qualquer lista.
- SEJA CONCISO: Mantenha cada item breve e direto ao ponto.
- VALIDE: Cada informação deve estar explicitamente declarada no texto; não faça suposições.

Por favor, extraia:
- Liste as informações do Documento

Texto do documento:
{context}

"""
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

# https://cog-openai-stage.openai.azure.com/openai/deployments/gpt-4o-mini/chat/completions?api-version=2024-08-01-preview
llm = AzureChatOpenAI(azure_deployment="gpt-4o-mini", api_version="2024-08-01-preview", temperature=0.1)

In [28]:
from typing import List
from datetime import datetime, date

class LocalidadeModel(BaseModel):
    """Localização da Parte"""
    cidade: str = Field(description="Cidade da Parte")
    estado: str = Field(description="Estado da Parte")
    regiao: str = Field(description="Região do estado da Parte. Possíveis valores: S - Sul, SE - Sudeste, CO - Centro Oeste, NE - Nordeste, N - Norte")

class SecuritizadoraEmissoraModel(BaseModel):
    """Parte nomeadas e qualificadas como Securitizadora ou Emissora"""
    cnpj: str = Field(description="Número do CNPJ da Securitizadora ou Emissora. Formatos permitidos: ##.###.###/####-##")
    nome: str = Field(description="Nome da empresa ou Razão Social da Securitizadora ou Emissora")

class AgenteFiduciarioModel(BaseModel):
    """Parte nomeadas e qualificadas como Agente Fiduciário"""
    cnpj: str = Field(description="Número do CNPJ do Agente Fiduciário. Formatos permitidos: ##.###.###/####-##")
    nome: str = Field(description="Nome da empresa ou Razão Social do Agente Fiduciário")

class DevedoraModel(BaseModel):
    """Parte nomeadas e qualificadas como Devedores ou Devedoras"""
    cpf_cnpj: str = Field(description="Número do CPF ou CNPJ do Devedor. Formatos permitidos: ##.###.###/####-## ou ###.###.###-##")
    nome: str = Field(description="Nome do Devedor")
    localidade: LocalidadeModel = Field(description="Localidade do Devedor")

class CedenteModel(BaseModel):
    """Parte nomeadas e qualificadas como Cedentes"""
    cpf_cnpj: str = Field(description="Número do CPF ou CNPJ do Cedente. Formatos permitidos: ##.###.###/####-## ou ###.###.###-##")
    nome: str = Field(description="Nome do Cedente")
    localidade: LocalidadeModel = Field(description="Localidade do Devedor")

class AvalistaFiadorModel(BaseModel):
    """Parte nomeadas e qualificadas como Avalista ou Fiador"""
    cpf_cnpj: str = Field(description="Número do CPF ou CNPJ do Avalista ou Fiador. Formatos permitidos: ##.###.###/####-## ou ###.###.###-##")
    nome: str = Field(description="Nome do Avalista ou Fiador")
    localidade: LocalidadeModel = Field(description="Localidade do Devedor")

class RemuneracaoModel(BaseModel):
    """Informações sobre a remuneração do Ativo"""
    indexador: str = Field(description="Indexador da Remuneração. Valores permitidos: IPCA, DI, Pré, INCC, PTAX, IGPM", examples=['IPCA', 'DI', 'Pré', 'INCC', 'PTAX', 'IGPM'])

    variacao: float = Field(description="Variação percentual acumulada do indexador da Remuneração em %", examples=[100])

    taxa_adicional_spread: float = Field(description="Taxa Adicional pré fixada ou Spread acrescida na Remuneração em %", examples=[1.0, 5.5, 10.0])

    periodo_taxa: str = Field(description="Periodo da Taxa Adicional. Valores permitidos: aa - para ao ano e am - para ao mês.", examples=['aa', 'am'])

class SerieModel(BaseModel):
    """Informações das Series"""
    numero_serie: str = Field(description="Número da série em sua forma apenas numérica. Caso seja uma série única, retorne UNI.", examples=['1', '2', '3', 'UNI'])

    classe_serie: str = Field(description="Classe da série. Valores permitidos: Senior ou Subordinada", examples=['Senior', 'Subordinada'])

    qt_serie: int = Field(description="Quantidade da Série")

    vlr_serie: float = Field(description="Valor da Série")

    remuneracao: RemuneracaoModel = Field(description="Remuneração da Série")

class CoordenadorLiderModel(BaseModel):
    """Parte nomeadas e qualificadas como Coordenador Lider"""
    cnpj: str = Field(description="Número do CNPJ do Coordenador Líder. Formatos permitidos: ##.###.###/####-##")
    nome: str = Field(description="Nome da empresa ou Razão Social do Coordenador Líder")

class ResponsavelGarantiaModel(BaseModel):
    """Parte nomeadas e qualificadas como Responsável"""
    cpf_cnpj: str = Field(description="Número do CPF ou CNPJ do Responsável da Garantia. Formatos permitidos: ##.###.###/####-## ou ###.###.###-##")
    nome: str = Field(description="Nome do Responsável da Garantia")

class GarantiaModel(BaseModel):
    """Informações das Garantias"""
    garantia: str = Field(description="""Nome da Garantia""")

    responsavel: ResponsavelGarantiaModel = Field(description="Responsavel Garantia")


In [12]:
class AssembleiaModel(BaseModel):
    """Informações importantes da Assembleia"""
    emissora: SecuritizadoraEmissoraModel = Field(description="Securitizadora ou Emissora")

    agente_fiduciario: AgenteFiduciarioModel = Field(description="Agente Fiduciário")

    tipo: str = Field(description="""
    Sendo CERTIFICADOS DE RECEBIVEIS DO AGRONEGOCIO = CRA;
    Sendo CERTIFICADOS DE RECEBIVEIS IMOBILIARIOS = CRI;
    Sendo DEBENTURES = DEB;
    Sendo CERTIFICADOS DE RECEBIVEIS = CR;
    Retorne se CRI, CRA, CR ou DEB. Caso não encontre, retorne ''.
    """)
    numero_emissao: str = Field(description="""
    Número da emissão em sua forma apenas numérica.
    """)
    numero_serie: List[str] = Field(description="""
    Número da série em sua forma apenas numérica.
    Caso seja uma série única, retorne UNI.
    """)
    dt_hr_assembleia: datetime = Field(description="data e hora da assembleia")
    numero_convocacao: int = Field(description="número de convocação da assembleia nesta data")
    fl_assembleia_deliberada: bool = Field(description="A assembleia teve a Ordem do Dia deliberada?")
    fl_assembleia_nao_instalada: bool = Field(description="A assembleia não foi instalada?")

In [29]:
class TermoSecModel(BaseModel):
    """Informações importantes do Termo de Securitização"""
    emissora: SecuritizadoraEmissoraModel = Field(description="Securitizadora ou Emissora")

    agente_fiduciario: AgenteFiduciarioModel = Field(description="Agente Fiduciário")

    devedores: List[DevedoraModel] = Field(description="Lista Única de Devedores ou Devedoras")

    cedentes: List[CedenteModel] = Field(description="Lista Única de Cedentes")

    avalistas_fiadores: List[AvalistaFiadorModel] = Field(description="Lista Única de Avalistas ou Fiadores")

    coordenador_lider: CoordenadorLiderModel = Field(description="Coordenador Lider")

    numero_aditamento: int = Field(description="Número do aditamento. Caso não seja um aditamento retorne 0")

    tipo: str = Field(description="""
    Sendo CERTIFICADOS DE RECEBIVEIS DO AGRONEGOCIO = CRA;
    Sendo CERTIFICADOS DE RECEBIVEIS IMOBILIARIOS = CRI;
    Sendo DEBENTURES = DEB;
    Sendo CERTIFICADOS DE RECEBIVEIS = CR;
    Retorne se CRI, CRA, CR ou DEB. Caso não encontre, retorne ''.
    """)
    numero_emissao: str = Field(description="""Número da emissão em sua forma apenas numérica.""")

    series: List[SerieModel] = Field(description="""Lista Única de Séries da Emissão.""")

    garantias: List[GarantiaModel] = Field(description="""Garantias da Emissão""")

    dt_emissao: date = Field(description="data da emissão")

    dt_vencimento: date = Field(description="data do vencimento")

    numero_quorum_minimo_primeira_convocacao: str = Field(description="Número quórum mínimo de instalação de assembleia em primeira convocação com a presença de Titulares ou Investidores em Circulação. Retorne o mais simples possível.")

    numero_quorum_minimo_segunda_convocacao: str = Field(description="Número quórum mínimo de instalação de assembleia em segunda convocação com a presença de Titulares ou Investidores em Circulação. Retorne o mais simples possível.")

    qt_total_emissao: int = Field(description="Quantidade Total da Emissão ou da Oferta")

    vlr_total_emissao: float = Field(description="Valor Total da Emissão ou da Oferta")

In [14]:
class EscrituraModel(BaseModel):
    """Informações importantes da Escritura"""
    emissora: SecuritizadoraEmissoraModel = Field(description="Securitizadora ou Emissora")

    agente_fiduciario: AgenteFiduciarioModel = Field(description="Agente Fiduciário")

    tipo: str = Field(description="""
    Sendo CERTIFICADOS DE RECEBIVEIS DO AGRONEGOCIO = CRA;
    Sendo CERTIFICADOS DE RECEBIVEIS IMOBILIARIOS = CRI;
    Sendo DEBENTURES = DEB;
    Sendo CERTIFICADOS DE RECEBIVEIS = CR;
    Retorne se CRI, CRA, CR ou DEB. Caso não encontre, retorne ''.
    """)
    numero_emissao: str = Field(description="""
    Número da emissão em sua forma apenas numérica.
    """)
    numero_serie: List[str] = Field(description="""
    Número da série em sua forma apenas numérica.
    Caso seja uma série única, retorne UNI.
    """)
    dt_emissao: date = Field(description="data da emissão")

    dt_vencimento: date = Field(description="data do vencimento")

    numero_quorum_minimo_primeira_convocacao: str = Field(description="Número quórum mínimo de instalação de assembleia em primeira convocação com a presença de Titulares ou Investidores em Circulação. Retorne o mais simples possível.")

    numero_quorum_minimo_segunda_convocacao: str = Field(description="Número quórum mínimo de instalação de assembleia em segunda convocação com a presença de Titulares ou Investidores em Circulação. Retorne o mais simples possível.")

In [15]:
def format_docs(docs):
    return "\n\n ------------".join(doc.page_content for doc in docs)

In [16]:
def invoke(model):
    llm_with_schema = llm.with_structured_output(model)

    chain = prompt | llm_with_schema

    result_llm = chain.invoke({"context": format_docs(docs), "input": "Extraia as informações"})
    return result_llm

## Teste Documentos de Assembleia

In [22]:
result = invoke(AssembleiaModel)
result

AssembleiaModel(emissora=SecuritizadoraEmissoraModel(cnpj='41.811.375/0001-19', nome='CANAL COMPANHIA DE SECURITIZAÇÃO'), agente_fiduciario=AgenteFiduciarioModel(cnpj='', nome=''), tipo='CRI', numero_emissao='29', numero_serie=['UNI'], dt_hr_assembleia=datetime.datetime(2025, 2, 12, 14, 0), numero_convocacao=2, fl_assembleia_deliberada=False, fl_assembleia_nao_instalada=False)

## Teste Documentos de Termo de Securitização

In [30]:
result = invoke(TermoSecModel)
result.model_dump_json(indent=3)

TermoSecModel(emissora=SecuritizadoraEmissoraModel(cnpj='41.534.746/0001-62', nome='CERES SECURITIZADORA S.A.'), agente_fiduciario=AgenteFiduciarioModel(cnpj='36.113.876/0004-34', nome='OLIVEIRA TRUST DISTRIBUIDORA DE TÍTULOS E VALORES MOBILIÁRIOS S.A.'), devedores=[DevedoraModel(cpf_cnpj='19.092.942/0001-75', nome='NATIVA AGRONEGÓCIOS LTDA.', localidade=LocalidadeModel(cidade='Sinop', estado='MT', regiao='CO'))], cedentes=[], avalistas_fiadores=[AvalistaFiadorModel(cpf_cnpj='045.063.786-79', nome='ADRIANO DA SILVA CAMPOS', localidade=LocalidadeModel(cidade='Sinop', estado='MT', regiao='CO')), AvalistaFiadorModel(cpf_cnpj='941.170.719-04', nome='EDERSON PILONETTO', localidade=LocalidadeModel(cidade='Sinop', estado='MT', regiao='CO')), AvalistaFiadorModel(cpf_cnpj='833.029.219-34', nome='SILVIO VALDIR KAFER', localidade=LocalidadeModel(cidade='Sinop', estado='MT', regiao='CO'))], coordenador_lider=CoordenadorLiderModel(cnpj='65.913.436/0001-17', nome='GUIDE INVESTIMENTOS S.A. CORRETORA 

In [31]:
result.series

[SerieModel(numero_serie='1', classe_serie='Senior', qt_serie=16800, vlr_serie=16800000.0, remuneracao=RemuneracaoModel(indexador='DI', variacao=100.0, taxa_adicional_spread=5.5, periodo_taxa='aa')),
 SerieModel(numero_serie='2', classe_serie='Subordinada', qt_serie=4200, vlr_serie=4200000.0, remuneracao=RemuneracaoModel(indexador='DI', variacao=100.0, taxa_adicional_spread=4.2, periodo_taxa='aa'))]

## Teste Documentos de Escritura de Debentures

In [16]:
result = invoke(EscrituraModel)
result

NotFoundError: Error code: 404 - {'error': {'code': '404', 'message': 'Resource not found'}}